## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests

from citipy import citipy
from datetime import datetime
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,59.90,73,100,1.88,overcast clouds
1,1,Carnarvon,AU,-24.8667,113.6333,75.27,50,0,26.46,clear sky
2,2,Batu Gajah,MY,4.4692,101.0411,80.62,76,91,3.13,overcast clouds
3,3,Iqaluit,CA,63.7506,-68.5145,-7.31,63,47,3.00,scattered clouds
4,4,Kruisfontein,ZA,-34.0033,24.7314,63.88,73,66,1.39,broken clouds
5,5,Nicoya,CR,10.1483,-85.4520,72.95,83,59,1.57,broken clouds
6,6,Plettenberg Bay,ZA,-34.0527,23.3716,61.56,81,23,2.19,few clouds
7,7,Leh,IN,34.1667,77.5833,2.73,48,81,2.95,broken clouds
8,8,Iquique,CL,-20.2208,-70.1431,66.24,76,1,4.76,clear sky
9,9,Cape Town,ZA,-33.9258,18.4232,63.09,83,99,5.23,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,59.90,73,100,1.88,overcast clouds
1,1,Carnarvon,AU,-24.8667,113.6333,75.27,50,0,26.46,clear sky
2,2,Batu Gajah,MY,4.4692,101.0411,80.62,76,91,3.13,overcast clouds
4,4,Kruisfontein,ZA,-34.0033,24.7314,63.88,73,66,1.39,broken clouds
5,5,Nicoya,CR,10.1483,-85.4520,72.95,83,59,1.57,broken clouds
6,6,Plettenberg Bay,ZA,-34.0527,23.3716,61.56,81,23,2.19,few clouds
8,8,Iquique,CL,-20.2208,-70.1431,66.24,76,1,4.76,clear sky
9,9,Cape Town,ZA,-33.9258,18.4232,63.09,83,99,5.23,overcast clouds
10,10,Albany,US,42.6001,-73.9662,64.47,85,100,1.01,moderate rain
11,11,East London,ZA,-33.0153,27.9116,70.05,81,71,3.04,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,59.90,73,100,1.88,overcast clouds
1,1,Carnarvon,AU,-24.8667,113.6333,75.27,50,0,26.46,clear sky
2,2,Batu Gajah,MY,4.4692,101.0411,80.62,76,91,3.13,overcast clouds
4,4,Kruisfontein,ZA,-34.0033,24.7314,63.88,73,66,1.39,broken clouds
5,5,Nicoya,CR,10.1483,-85.4520,72.95,83,59,1.57,broken clouds
...,...,...,...,...,...,...,...,...,...,...
641,641,Tanete,ID,-3.9443,119.7848,77.07,92,100,5.73,overcast clouds
642,642,Plato,CO,9.7903,-74.7824,75.58,85,100,3.56,overcast clouds
643,643,Pailon,BO,-17.6500,-62.7500,76.10,83,100,6.35,overcast clouds
644,644,Yeppoon,AU,-23.1333,150.7333,87.58,61,32,17.85,scattered clouds


In [8]:
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hermanus,ZA,59.90,overcast clouds,-34.4187,19.2345,
1,Carnarvon,AU,75.27,clear sky,-24.8667,113.6333,
2,Batu Gajah,MY,80.62,overcast clouds,4.4692,101.0411,
4,Kruisfontein,ZA,63.88,broken clouds,-34.0033,24.7314,
5,Nicoya,CR,72.95,broken clouds,10.1483,-85.4520,
6,Plettenberg Bay,ZA,61.56,few clouds,-34.0527,23.3716,
8,Iquique,CL,66.24,clear sky,-20.2208,-70.1431,
9,Cape Town,ZA,63.09,overcast clouds,-33.9258,18.4232,
10,Albany,US,64.47,moderate rain,42.6001,-73.9662,
11,East London,ZA,70.05,broken clouds,-33.0153,27.9116,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hermanus,ZA,59.90,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
1,Carnarvon,AU,75.27,clear sky,-24.8667,113.6333,Hospitality Carnarvon
2,Batu Gajah,MY,80.62,overcast clouds,4.4692,101.0411,Pusing Paradise Homestay
4,Kruisfontein,ZA,63.88,broken clouds,-34.0033,24.7314,Oyster Bay House Rental
5,Nicoya,CR,72.95,broken clouds,10.1483,-85.4520,Las Cabinas Del Sueño


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))